In [1]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

torch.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "gpt2"  # Change this to "gpt2" if you get memory issues

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            return_dict_in_generate=True,
                                            pad_token_id=tokenizer.eos_token_id).to(device)

In [2]:
prompt = "The"
input_ids = torch.tensor([tokenizer.bos_token_id,] + tokenizer.encode(prompt)).reshape(1, -1).to(device)

with torch.no_grad():
    output = model(input_ids)
    logits = output[0]
    probs = logits.softmax(-1)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


In [3]:
print(probs.shape)

torch.Size([1, 2, 50257])


In [4]:
index = tokenizer.encode("English")
for i in range(probs.shape[-1]):
    prob = probs[0, -1, i].item()
    # if prob > 0.0001:
    print(f"Probability of {tokenizer.decode(i)} ({i}) is {prob:.4f}")

Probability of ! (0) is 0.0000
Probability of " (1) is 0.0000
Probability of # (2) is 0.0000
Probability of $ (3) is 0.0000
Probability of % (4) is 0.0000
Probability of & (5) is 0.0000
Probability of ' (6) is 0.0000
Probability of ( (7) is 0.0000
Probability of ) (8) is 0.0000
Probability of * (9) is 0.0000
Probability of + (10) is 0.0000
Probability of , (11) is 0.0001
Probability of - (12) is 0.0001
Probability of . (13) is 0.0000
Probability of / (14) is 0.0000
Probability of 0 (15) is 0.0000
Probability of 1 (16) is 0.0000
Probability of 2 (17) is 0.0000
Probability of 3 (18) is 0.0000
Probability of 4 (19) is 0.0000
Probability of 5 (20) is 0.0000
Probability of 6 (21) is 0.0000
Probability of 7 (22) is 0.0000
Probability of 8 (23) is 0.0000
Probability of 9 (24) is 0.0000
Probability of : (25) is 0.0000
Probability of ; (26) is 0.0000
Probability of < (27) is 0.0000
Probability of = (28) is 0.0000
Probability of > (29) is 0.0000
Probability of ? (30) is 0.0000
Probability of @ (

Probability of rop (1773) is 0.0000
Probability of  proper (1774) is 0.0000
Probability of  seen (1775) is 0.0000
Probability of ); (1776) is 0.0000
Probability of  fac (1777) is 0.0000
Probability of  Su (1778) is 0.0000
Probability of  cond (1779) is 0.0000
Probability of iting (1780) is 0.0000
Probability of  course (1781) is 0.0001
Probability of  } (1782) is 0.0000
Probability of ---------------- (1783) is 0.0000
Probability of aign (1784) is 0.0000
Probability of  event (1785) is 0.0005
Probability of  eng (1786) is 0.0000
Probability of  pot (1787) is 0.0000
Probability of  intern (1788) is 0.0000
Probability of iam (1789) is 0.0000
Probability of  short (1790) is 0.0004
Probability of empt (1791) is 0.0000
Probability of � (1792) is 0.0000
Probability of  God (1793) is 0.0001
Probability of ilar (1794) is 0.0000
Probability of 80 (1795) is 0.0000
Probability of  orig (1796) is 0.0000
Probability of IS (1797) is 0.0000
Probability of ourn (1798) is 0.0000
Probability of ability 

In [6]:
words = ["medicine", "science"]
indexes = [tokenizer.encode(word) for word in words]

for i in indexes:
    word_prob = torch.prod(torch.stack([probs[0, -1, idx] for idx in i]))
    if prob > 0.0000:
        print(f"Probability of {tokenizer.decode(i)} is {word_prob:.4f}")

# for i in indexes:
#     prob = probs[0, -1, i].item()
#     if prob > 0.0000:
#         print(f"Probability of {tokenizer.decode(i)} is {prob:.4f}")

Probability of medicine is 0.0000
Probability of science is 0.0000


In [7]:
words_probs = {}

for i in range(probs.shape[-1]):
    prob = probs[0, -1, i].item()
    if prob > 0.0001:
        words_probs[(tokenizer.decode(i)).replace(" ", "")] = prob

In [8]:
wanted_words_probs = {}
for word in words:
    wanted_words_probs[word] = words_probs[word]

# Apply softmax to the probabilities in wanted_words_probs
total = sum(wanted_words_probs.values())
for word in wanted_words_probs:
    wanted_words_probs[word] = wanted_words_probs[word] / total

print(wanted_words_probs)

{'medicine': 0.7293134914093928, 'science': 0.2706865085906071}


In [2]:
prompt = "The "
input_ids = torch.tensor([tokenizer.bos_token_id,] + tokenizer.encode(prompt)).reshape(1, -1).to(device)

with torch.no_grad():
    output = model(input_ids)
    logits = output[0]
    probs = logits.softmax(-1)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


In [3]:
words_probs = {}

for i in range(probs.shape[-1]):
    prob = probs[0, -1, i].item()
    # if prob > 0.0001:
    words_probs[(tokenizer.decode(i)).replace(" ", "")] = prob

print(words_probs)

{'!': 1.7316970115643926e-06, '"': 3.640041541075334e-06, '#': 1.6806415032988298e-06, '$': 3.623679276643088e-06, '%': 6.393075864252751e-07, '&': 4.4069355453757453e-07, "'": 1.6337564829882467e-06, '(': 1.004488058242714e-05, ')': 4.896527343589696e-07, '*': 1.7410429791198112e-06, '+': 3.4822386396626825e-07, ',': 7.8925586421974e-07, '-': 1.1914532933587907e-06, '.': 1.260626049770508e-06, '/': 1.2610389887868223e-07, '0': 6.629185236306512e-07, '1': 6.819037139393913e-07, '2': 3.2032929198066995e-07, '3': 6.162108547869138e-07, '4': 2.1999602495270665e-07, '5': 4.0787978150547133e-07, '6': 9.778141674132712e-08, '7': 4.382342524422711e-07, '8': 4.7606002340216946e-07, '9': 2.5942836145986803e-07, ':': 7.157912023103563e-07, ';': 5.611498181679053e-07, '<': 1.008538333735487e-06, '=': 2.8935160116816405e-06, '>': 3.0775228765378415e-07, '?': 5.541787118090724e-07, '@': 2.23523647946422e-06, 'A': 7.589243864458695e-07, 'B': 4.5034821027911676e-07, 'C': 2.544008737004333e-07, 'D': 7

In [4]:
man_woman = ["man", "woman"]
man_woman_probs = {}
for word in man_woman:
    man_woman_probs[word] = words_probs[word]

# Apply softmax to the probabilities in man_woman_probs
total = sum(man_woman_probs.values())
for word in man_woman_probs:
    man_woman_probs[word] = man_woman_probs[word] / total

print(man_woman_probs)

{'man': 0.863001508890443, 'woman': 0.13699849110955692}


In [27]:
import random

for i in range(10):
    man_woman = random.choices(list(man_woman_probs.keys()), weights=list(man_woman_probs.values()))[0]
    job = random.choices(list(wanted_words_probs.keys()), weights=list(wanted_words_probs.values()))[0]
    print(f"The {man_woman} was trained in {job}")


The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in science
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine


In [5]:
import random

man_woman = ["man", "woman"]
jobs = ["medicine", "science"]

prompt = "The"
input_ids = torch.tensor([tokenizer.bos_token_id,] + tokenizer.encode(prompt)).reshape(1, -1).to(device)

with torch.no_grad():
    output = model(input_ids)
    logits = output[0]
    probs = logits.softmax(-1)
words_probs = {}

for i in range(probs.shape[-1]):
    prob = probs[0, -1, i].item()
    # if prob > 0.0001:
    words_probs[(tokenizer.decode(i)).replace(" ", "")] = prob

man_woman_probs = {}
for word in man_woman:
    man_woman_probs[word] = words_probs[word]

# Apply softmax to the probabilities in man_woman_probs
total = sum(man_woman_probs.values())
for word in man_woman_probs:
    man_woman_probs[word] = man_woman_probs[word] / total

man_prompt = "The man was trained in"
input_ids = torch.tensor([tokenizer.bos_token_id,] + tokenizer.encode(prompt)).reshape(1, -1).to(device)

with torch.no_grad():
    output = model(input_ids)
    logits = output[0]
    probs = logits.softmax(-1)
words_probs = {}

for i in range(probs.shape[-1]):
    prob = probs[0, -1, i].item()
    # if prob > 0.0001:
    words_probs[(tokenizer.decode(i)).replace(" ", "")] = prob

man_jobs_probs = {}
for word in jobs:
    man_jobs_probs[word] = words_probs[word]

woman_prompt = "The woman was trained in"
input_ids = torch.tensor([tokenizer.bos_token_id,] + tokenizer.encode(prompt)).reshape(1, -1).to(device)

with torch.no_grad():
    output = model(input_ids)
    logits = output[0]
    probs = logits.softmax(-1)
words_probs = {}

for i in range(probs.shape[-1]):
    prob = probs[0, -1, i].item()
    # if prob > 0.0001:
    words_probs[(tokenizer.decode(i)).replace(" ", "")] = prob

woman_jobs_probs = {}
for word in jobs:
    woman_jobs_probs[word] = words_probs[word]

for i in range(20):
    man_woman = random.choices(list(man_woman_probs.keys()), weights=list(man_woman_probs.values()))[0]
    if man_woman == "man":
        job = random.choices(list(man_jobs_probs.keys()), weights=list(man_jobs_probs.values()))[0]
    elif man_woman == "woman":
        job = random.choices(list(woman_jobs_probs.keys()), weights=list(woman_jobs_probs.values()))[0]
    else:
        print("Error")
        
    print(f"The {man_woman} was trained in {job}")


The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The woman was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
The man was trained in medicine
